In [1]:
import os
cwd = os.getcwd()
print(cwd)

C:\Users\feder\Documents\GitHub\relgan\src\jupyter_tests


In [2]:
os.chdir("..")
cwd = os.getcwd()
print(cwd)
assert cwd.endswith("src")

C:\Users\feder\Documents\GitHub\relgan\src


# Data Analysis on the Amazon Attribute dataset

In [25]:
import pandas as pd
import numpy as np
from os.path import join
import nltk
from tqdm import tqdm
from utils.text_process import get_word_list, get_dict, text_to_code
from path_resolution import resources_path
from utils.static_file_manage import write_json

In [26]:
data_dir = resources_path("data", "Amazon_Attribute")

### Dataset import

In [27]:
data = pd.read_csv(join(data_dir, 'full_datasets', 'train.txt'), sep="\t", header=None)
data.columns = ["user_id", "product_id", "rating", "review"]

In [28]:
data[data.user_id == 8478]

,user_id,product_id,rating,review
1342,8478,28638,5.0,i will always love parker . this shows a diffe...
43173,8478,42105,4.0,"having read almost all of his books , it 's am..."
147911,8478,17224,3.0,"i like parkers books , this one was OK , but h..."
243700,8478,7927,5.0,this was a different spenser mystery . rather ...
258711,8478,34335,5.0,i simply like parker 's terse style . ca n't t...
357798,8478,60982,5.0,really loved this spenser mystery . the storyl...
369755,8478,79101,4.0,this book show the heart of spenser . he found...
473596,8478,37372,4.0,i 'll write enough for you to accept the ratin...
618346,8478,32395,4.0,"typical , but good spenser novel . this one in..."
655087,8478,33312,5.0,i 've read this great book three times over th...


#### Compute product_ids and user_ids to remove because they occur too few times

In [29]:
product_count = data['product_id'].value_counts()
product_id_keep = product_count[product_count.apply(lambda x: x>15)].index

user_count = data['user_id'].value_counts()
user_id_keep = user_count[user_count.apply(lambda x: x>6)].index

In [30]:
def remove_low_index(df, user_id_keep, product_id_keep):
    df = df[df['user_id'].isin(user_id_keep)]
    return df[df.product_id.isin(product_id_keep)]

from collections import Counter 
def removeElements(lst, k): 
    counted = Counter(lst) 
    return [el for el in lst if counted[el] >= k] 

In [52]:
token_list = []
dataframes = []
file_names = ['train.txt', 'test.txt', 'dev.txt']
for file_name in file_names:
    data_tmp = pd.read_csv(join(data_dir, 'full_datasets',file_name), sep="\t", header=None)
    data_tmp.columns = ["user_id", "product_id", "rating", "review"]
    data_tmp = remove_low_index(data_tmp, user_id_keep, product_id_keep)
    for text in tqdm(data_tmp.review):
        text = nltk.word_tokenize(text.lower())
        token_list.append(text)
    dataframes.append(data_tmp)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22644/22644 [00:07<00:00, 3119.48it/s]


In [10]:
token_list

[['love', 'this', 'book', 'too', 'have', 'read', 'it', 'again'],
 ['i', 'really', 'enjoyed', 'this', 'book', '.', ':', ')'],
 ['fast', 'paced', 'and', 'a', 'load', 'of', 'action', '.'],
 ['fun', ',', 'somewhat', 'predictable', ',', 'but', 'very', 'enjoyable'],
 ['great',
  'series.love',
  'hunter',
  'my',
  'favorite',
  'boyfriend',
  'character',
  '.'],
 ['another', 'very', 'enjoyable', 'addition', 'to', 'the', 'series', '.'],
 ['it', "'s", 'been', 'too', 'long', '.', 'please', 'delete'],
 ['<', 'num', '>', '1/2', 'stars', '.', 'love', 'these', 'books', '!'],
 ['i', 'enjoyed', 'this', 'book', '.', 'good', 'read', '.'],
 ['hot', 'book', '!', 'i', 'love', 'roz', 'lee', '!'],
 ['i', 'do', "n't", 'remember', 'what', 'this', 'is', 'about'],
 ['i', 'ca', "n't", 'remember', 'what', 'it', "'s", 'about'],
 ['always', 'a', 'must', 'in', 'the', 'reference', 'library', '.'],
 ['i', 'ca', "n't", 'remember', 'what', 'it', 'is', 'about'],
 ['i', 'ca', "n't", 'remember', 'what', 'this', 'is', 'ab

In [53]:
print(len(token_list))
token_list_removed = removeElements([x for sent in token_list for x in sent], 11)
print(len(token_list))

word_set = list(dict.fromkeys(set(token_list_removed)))
[word_index_dict, index_word_dict] = get_dict(word_set)
sequence_len = len(max(token_list, key=len))
    
    
# data.to_csv(join(data_dir, file_name[:-4] + '.csv'), sep='\t')

234021
234021


In [12]:
len(word_set)

11212

In [21]:
df = dataframes[0]
sorted_users = np.sort(df.user_id.unique())
user_remap = dict(zip(sorted_users, range(len(sorted_users))))
sorted_products = np.sort(df.product_id.unique())
product_remap = dict(zip(sorted_products, range(len(sorted_products))))

In [28]:
df.head()

,user_id,product_id,rating,review,tokenized_text
6,9200,77458,5.0,love this book too have read it again,0 1 2 3 4 5 6 7 80611 80611 80611 80611 80611 ...
14,10646,49714,5.0,i really enjoyed this book . : ),8 9 10 1 2 11 12 13 80611 80611 80611 80611 80...
37,7995,11002,5.0,fast paced and a load of action .,14 15 16 17 18 19 20 11 80611 80611 80611 8061...
46,1707,36253,4.0,"fun , somewhat predictable , but very enjoyable",21 22 23 24 22 25 26 27 80611 80611 80611 8061...
76,15531,22206,5.0,great series.love hunter my favorite boyfriend...,28 29 30 31 32 33 34 11 80611 80611 80611 8061...


In [13]:
def apply_text_to_code(x):
    tokenization = nltk.word_tokenize(x.lower())
    eof_code = len(word_index_dict)  # used to filled in the blank to make up a sentence with seq_len
    index = 0
    code_int = []
    for word in tokenization:
        try:
            code_int.append(int(word_index_dict[word]))
        except KeyError:
            return np.nan
        index += 1
    while index < sequence_len:
        code_int.append(int(eof_code))
        index += 1
    return code_int
    

In [54]:
for df, file_name in tqdm(zip(dataframes, file_names)):
    df['tokenized_text'] = df['review'].apply(lambda x: apply_text_to_code(x))

In [62]:
for df in dataframes:
    df.dropna(inplace=True)

In [64]:
df = dataframes[0]
sorted_users = np.sort(df.user_id.unique())
user_remap = dict(zip(sorted_users, range(len(sorted_users))))
sorted_products = np.sort(df.product_id.unique())
product_remap = dict(zip(sorted_products, range(len(sorted_products))))

In [65]:
for df, file_name in zip(dataframes, file_names):
    df.user_id = df.user_id.map(user_remap)
    df.product_id = df.product_id.map(product_remap)

## Per ora ho eliminato le frasi che contenevano una parola che appariva meno di 10 volte. Non ho usato l'out of vocabulary

In [51]:
len(data)

655934

In [60]:
a = dataframes[0].dropna()

In [61]:
len(a)

95296

In [71]:
dataframes[0][dataframes[0]['user_id'] == 13852]

,user_id,product_id,rating,review,tokenized_text
195473,13852,3915,5.0,could n't put it down . based on a true story ...,"[4582, 1111, 8227, 5888, 4708, 1794, 7612, 565..."
649405,13852,1563,4.0,"i read the book and historical it was good , w...","[2713, 8630, 1984, 6997, 9293, 6934, 5888, 103..."


In [72]:
word_index_dict['i']

'2713'

In [73]:
index_word_dict['2713']

'i'

In [68]:
for df in dataframes:
    df.reset_index()

In [69]:
for df, file_name in zip(dataframes, file_names):
    df.to_csv(join(data_dir, file_name[:-4] + '.csv'))

In [74]:
config_dict = {}
config_dict['seq_len'] = sequence_len
config_dict['vocabulary_size'] = len(word_index_dict) + 1
config_dict['user_num'] = len(sorted_users)
config_dict['product_num'] = len(sorted_products)

write_json(join(data_dir, 'word_index_dict.json'), word_index_dict)
write_json(join(data_dir, 'index_word_dict.json'), index_word_dict)
write_json(join(data_dir, 'config.json'), config_dict)


In [15]:
batch_size = 64
df = dataframes[0]
token_stream = df[['user_id', 'product_id', 'rating', 'tokenized_text']].values

In [34]:
dataframes[1].head()

,user_id,product_id,rating,review,tokenized_text
2,2225.0,4065,5.0,really like all of the joe dillard series,"[7726, 5563, 5734, 10891, 4671, 9420, 10743, 7..."
3,7436.0,2744,1.0,pretty terrible . at least it was ago,"[6446, 10528, 3032, 6325, 6456, 5276, 245, 686..."
14,4582.0,822,5.0,excellent read ! will continue the series .,"[8246, 5744, 7398, 2755, 9035, 4671, 7771, 303..."
20,7114.0,2937,5.0,"omg , this was a great read .","[9320, 9923, 3951, 245, 154, 3921, 5744, 3032,..."
27,10416.0,4661,5.0,really cute . loved it start to finish,"[7726, 5330, 3032, 624, 5276, 8435, 5312, 2344..."


In [17]:
num_batch = int(len(token_stream) / batch_size)
token_stream = token_stream[:num_batch * batch_size]
sequence_batches = np.split(np.array(token_stream), num_batch, axis=0)

In [19]:
sequence_batches[0][0]

array([9200, 77458, 5.0,
       '0 1 2 3 4 5 6 7 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 80611 \n'],
      dtype=object)